In [74]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
%run /home/datarian/git/master-thesis-code/notebooks/common_init.ipynb

Setup logging to file: out.log
Figure output directory saved in figure_output at /home/datarian/OneDrive/unine/Master_Thesis/ma-thesis-report/figures


In [76]:
%run /home/datarian/git/master-thesis-code/notebooks/learning_init.ipynb

Set plot_confusion_matrix()


In [77]:
%autoreload 2
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# Load custom code
import kdd98.data_handler as dh
from kdd98.config import Config
import pathlib
import pickle

In [78]:
from collections import Counter
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [79]:
from sklearn.model_selection import ParameterGrid, GridSearchCV, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, make_scorer, log_loss, roc_auc_score, recall_score, precision_score, classification_report, roc_curve, auc
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [80]:
from scipy.stats import randint as sp_rint
from scipy.stats import uniform as sp_unif
from scipy.stats import bernoulli as sp_bern
from scipy.stats import rv_discrete as sp_discr
import numpy as np
np.random.seed(seed=Config.get("random_seed"))

In [81]:
# Where to save the figures

CHAPTER_ID = "gradient_boosting"
# Where to save the figures
IMAGES_PATH = pathlib.Path(figure_output/'eda')

pathlib.Path(IMAGES_PATH).mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension=["pdf", "png"], resolution=300):
    if tight_layout:
        plt.tight_layout()
    [plt.savefig(pathlib.Path(IMAGES_PATH, fig_id + "." + f), 
                 format=f,
                 dpi=resolution,
                 transparent=True,
                 bbox_inches='tight') for f in fig_extension]

In [82]:
with open(pathlib.Path(Config.get("df_store"), "X_train.pd.pkl"), "rb") as f:
    kdd98_learn_feat = pickle.load(f)
with open(pathlib.Path(Config.get("df_store"), "y_train.pd.pkl"), "rb") as f:
    kdd98_learn_targets = pickle.load(f)
with open(pathlib.Path(Config.get("df_store"), "X_test.pd.pkl"), "rb") as f:
    kdd98_test_feat = pickle.load(f)
with open(pathlib.Path(Config.get("df_store"), "y_test.pd.pkl"), "rb") as f:
    kdd98_test_targets = pickle.load(f)
    
# Extracting the data and resetting target to [-1, 1]
X_train = kdd98_learn_feat.values
y_train = kdd98_learn_targets.loc[:,"TARGET_B"].astype("int64").values
X_test = kdd98_test_feat.values
y_test = kdd98_test_targets.loc[:,"TARGET_B"].astype("int64").values

FileNotFoundError: [Errno 2] No such file or directory: '/data/home/datarian/git/master-thesis-code/data/data_frames/X_test.pd.pkl'

In [ ]:
class_weight = len(y_train[y_train == 0])/sum(y_train)

#  XGBoost

All fits are run with the dataset filtered down to all-relevant features with Boruta.

Parameters:

- J (tree size): max_depth. Hastie recommends 6, stumps (2) can be very good (only first-order), 4 < J <8
- M (number of trees): n_estimators
- $\nu$ (shrinkage): learning_rate: Small $\nu$ need large M
- $\eta$ (subsampling): colsample_by_tree, set to 1/2. This is effectively stochastic gradient descent
-> Set $\nu$ small and terminate by early stopping

early_stopping_rounds (int) – Activates early stopping. Validation error needs to decrease at least every <early_stopping_rounds> round(s) to continue training. Requires at least one item in evals. If there’s more than one, will use the last. Returns the model from the last iteration (not the best one). If early stopping occurs, the model will have three additional fields: bst.best_score, bst.best_iteration and bst.best_ntree_limit

## Baseline

In [ ]:
xgb_base = XGBClassifier(scale=class_weight,
                         seed=Config.get("random_seed"))
xgb_base.fit(X_train_all_relevant, y_train)

In [ ]:
y_predicted = xgb_base.predict(X_test_all_relevant)

In [ ]:
print(classification_report(y_test,y_predicted))

## Gridsearch with oversampled minority class
This approach uses an oversampled minority class

### Balancing target
Run either one of the below cells to get a sample with balanced target classes

In [ ]:
smt = SMOTETomek(sampling_strategy='minority',
                 random_state=Config.get("random_seed"))
X_resampled, y_resampled = smt.fit_resample(X_train_all_relevant, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

In [ ]:
ovs = RandomOverSampler(sampling_strategy="minority", random_state=Config.get("random_seed"))
X_resampled, y_resampled = ovs.fit_resample(X_train_all_relevant, y_train)
print(sorted(Counter(y_resampled).items()))

In [ ]:
smt = SMOTETomek(sampling_strategy='minority',
                 random_state=Config.get("random_seed"))
X_resampled, y_resampled = smt.fit_resample(X_train, train_target)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

### Setting up gridsearch

In [ ]:
# We use stochastic gradient boosting (subsample < 1),
# a maximum depth of 2 or 3,
# a very small learning rate combined with early stopping if there's no improvement
param_grid = {
    "learning_rate": [0.01, 0.05, 0.1],
    "colsample_bytree": [0.1, 0.2],
    "max_depth": [4,6,8,12]
}

# Scorer, we choose ROC and set weighted average according to class frequencies
roc = make_scorer(roc_auc_score)

xgb_oversampling = XGBClassifier(
    booster="gbtree",
    n_estimators=2000,
    learning_rate=0.1,
    subsample=0.5,
    max_depth=6,
    seed=Config.get("random_seed"),
    verbose=2)

In [ ]:
gs_xgboost_oversampling = GridSearchCV(
    xgb_oversampling,
    param_grid,
    scoring={
        "ROC_AUC": roc,
        "logloss": "neg_log_loss",
        "recall": make_scorer(recall_score)
    },
    n_jobs=-1,
    refit="recall",
    cv=5,
    verbose=3)

We pass an eval set. The last (X_test) is used for early stopping. When the eval metric does no longer decrease, training is stopped

### Fitting and results

In [ ]:
gs_xgboost_oversampling.fit(
    X_resampled,
    y_resampled,
    early_stopping_rounds=10,
    eval_metric="logloss",
    eval_set=[(X_resampled, y_resampled), (X_test_all_relevant, y_test)])

In [ ]:
with open(pathlib.Path(Config.get("model_store"),"gradient_boosting_boruta_oversampling.pkl"),"wb") as o:
    pickle.dump(gs_xgboost_oversampling, o)

In [ ]:
#with open(pathlib.Path(Config.get("model_store"),"gradient_boosting_boruta_oversampling.pkl"),"rb") as f:
#    gs_xgboost_oversampling = pickle.load(f)

In [ ]:
results = gs_xgboost_oversampling.cv_results_

In [ ]:
y_predict = gs_xgboost_oversampling.predict(X_test_all_relevant)

In [ ]:
print(classification_report(y_test,y_predict))

In [ ]:
plot_confusion_matrix(y_test, y_predict, [1,0], normalize=True, cmap=Config.get("color_map"))

In [ ]:
gs_xgboost_oversampling.best_params_

In [ ]:
best_estimator = gs_xgboost_oversampling.best_estimator_

In [ ]:
eval_results = gs_xgboost_oversampling.best_estimator_.evals_result()
epochs = len(eval_results['validation_0']['logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, eval_results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, eval_results['validation_1']['logloss'], label='Test')
logl_min = min(eval_results['validation_1']['logloss'])
logl_index = eval_results['validation_1']['logloss'].index(logl_min)
ax.plot((logl_index, logl_index), (0, logl_min), 'k-')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

In [ ]:
gs_xgboost_oversampling.predict_proba(X_test_all_relevant)

In [ ]:
# Compute fpr, tpr, thresholds and roc auc
fpr, tpr, thresholds = roc_curve(y_test, gs_xgboost_oversampling.predict_proba(X_test_all_relevant)[:,1])
roc_auc = auc(y_test, y_predict)

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

In [ ]:
del gs_xgboot

## Train on unbalanced data

Instead of oversampling the minority class, we put weights on the minority (positive) class. XGBoost then fits using these weights.

### Setting up gridsearch

In [ ]:
# Creating CV-folds respecting the class frequencies
cv = StratifiedShuffleSplit(
    n_splits=5, random_state=Config.get("random_seed"))

xgb_cls_weights = XGBClassifier(
    booster="gbtree",
    max_depth=6,
    subsample=0.5,
    scale_pos_weight=class_weight,
    n_estimators=2000,
    seed=Config.get("random_seed"),
    verbose=2,
    silent=False)

In [ ]:
param_grid = {
    "learning_rate": [0.005, 0.01, 0.05],
    "colsample_bytree": [0.1, 0.2, 0.3],
    "max_depth": [4,6]
}

gs_xgboost_class_weights = GridSearchCV(
    xgb_cls_weights,
    param_grid,
    scoring={
        "ROC_AUC": make_scorer(roc_auc_score),
        "logloss": "neg_log_loss",
        "recall": make_scorer(recall_score)
    },
    n_jobs=-1,
    cv=cv,
    refit="recall",
    verbose=3,
    return_train_score=True)

### Fitting and results

In [ ]:
gs_xgboost_class_weights.fit(
    X_train_all_relevant,
    y_train,
    early_stopping_rounds=10,
    eval_metric="logloss",
    eval_set=[(X_train_all_relevant, y_train), (X_test_all_relevant, y_test)])

In [ ]:
pd.DataFrame(gs_xgboost_class_weights.cv_results_)

In [ ]:
y_predict = gs_xgboost_class_weights.predict(X_test_all_relevant)

In [ ]:
print(classification_report(y_test,y_predict))

In [ ]:
plot_confusion_matrix(y_test, y_predict, [1,0], normalize=True, cmap=Config.get("color_map"))

In [ ]:
gs_xgboost_class_weights.best_estimator_

In [ ]:
eval_results = gs_xgboost_class_weights.best_estimator_.evals_result()
epochs = len(eval_results['validation_0']['logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, eval_results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, eval_results['validation_1']['logloss'], label='Test')
logl_min = min(eval_results['validation_1']['logloss'])
logl_index = eval_results['validation_1']['logloss'].index(logl_min)
ax.plot((logl_index, logl_index), (0, logl_min), 'k-')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

In [ ]:
with open(pathlib.Path(Config.get("model_store"),"gradient_boosting_boruta_classweights.pkl"),"rb") as of:
    gs_xgboost_class_weights = pickle.load(of)

In [ ]:
best_estimator = gs_xgboost_class_weights.best_estimator_

In [ ]:
del gs_xgboost_class_weights

##  Train with missing values on whole data set

In [10]:
provider = dh.KDD98DataProvider("cup98LRN.txt")

In [11]:
numeric = provider.numeric_data

In [13]:
seed = Config.get("random_seed")
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size=0.8, random_state=seed)
for learn_index, test_index in splitter.split(numeric, numeric.TARGET_B.astype('int')):
    train = numeric.iloc[learn_index]
    test = numeric.iloc[test_index]

In [14]:
X_train = train.drop(['TARGET_B', 'TARGET_D'],axis=1).copy()
y_train = train[['TARGET_B']].copy()

X_test = test.drop(['TARGET_B', 'TARGET_D'],axis=1).copy()
y_test = test[['TARGET_B']].astype("int").copy()
y_test = y_test.astype("int")

In [15]:
class_weight = len(y_train.values[y_train.values == 0])/sum(y_train.values)

In [16]:
class_weight

array([18.70289107])

In [17]:
class_weight = class_weight[0]

In [24]:
# We use stochastic gradient boosting (subsample < 1),
# a maximum depth of 2 or 3,
# a very small learning rate combined with early stopping if there's no improvement
param_grid = {
    "learning_rate": sp_unif(loc=0.01, scale=0.09),
    "max_depth": sp_rint(2, 5)
}

metrics = {
    "logloss": "neg_log_loss",
    "ROC_AUC": make_scorer(roc_auc_score),
    "recall": make_scorer(recall_score)
}

xgb_classifier = XGBClassifier(
    booster="gbtree",
    eval_metric=["logloss", "auc", "error"],
    scale_pos_weight=class_weight,
    n_estimators=2500,
    colsample_bytree=0.5,
    subsample=0.5,  # use half of features for each forest
    seed=Config.get("random_seed"),
    verbose=3,
    silent=False)

# Creating CV-folds respecting the class frequencies
cv = StratifiedShuffleSplit(n_splits=5, random_state=Config.get("random_seed"))

### Setting up gridsearch

In [25]:
gs_xgboost_with_missing = RandomizedSearchCV(
    xgb_classifier,
    param_grid,
    scoring=metrics,
    n_jobs=6,
    cv=cv,
    refit="recall",
    verbose=3,
    return_train_score=True)

### Fitting and results 

In [26]:
gs_xgboost_with_missing.fit(X_train.values, y_train.values, eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGABRT(-6)}

In [ ]:
gs_xgboost_with_missing.best_estimator_

In [ ]:
eval_results_missing = gs_xgboost_with_missing.best_estimator_.evals_result()
epochs = len(eval_results_missing['validation_0']['logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
#logloss
ax.plot(x_axis, eval_results_missing['validation_0']['logloss'], label='Train logloss')
ax.plot(x_axis, eval_results_missing['validation_1']['logloss'], label='Test logloss')
logl_min = min(eval_results_missing['validation_1']['logloss'])
logl_index = eval_results_missing['validation_1']['logloss'].index(logl_min)
ax.plot((logl_index, logl_index), (0, logl_min), 'k-')
#error
ax.plot(x_axis, eval_results_missing['validation_0']['logloss'], label='Train error')
ax.plot(x_axis, eval_results_missing['validation_1']['error'], label='Test error')

error_min = min(eval_results_missing['validation_1']['error'])
error_index = eval_results_missing['validation_1']['error'].index(error_min)
ax.plot((error_index, error_index), (0, error_min), 'k-')
#auc
ax.plot(x_axis, eval_results_missing['validation_0']['auc'], label='Train auc')
ax.plot(x_axis, eval_results_missing['validation_1']['auc'], label='Test auc')
auc_min = min(eval_results_missing['validation_1']['auc'])
auc_index = eval_results_missing['validation_1']['auc'].index(auc_min)
ax.plot((auc_index, auc_index), (0, auc_min), 'k-')

ax.legend()
plt.ylabel('Loss')
plt.title('XGBoost evaluation metrics')
plt.show()

In [ ]:
gs_xgboost_with_missing.best_params_

In [ ]:
plot_confusion_matrix(y_test, gs_xgboost_with_missing.best_estimator_.predict(X_test), [1,0], normalize=True)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, gs_xgboost_with_missing.best_estimator_.predict_proba(X_test)[:,1],pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")

In [ ]:
import xgboost
xgboost.plot_importance(gs_xgboost_with_missing.best_estimator_.get_booster(),
                        max_num_features=30,
                        show_values=True)

In [102]:
with open(pathlib.Path(Config.get("model_store"),"gridsearch_GBM_with_missing_values.pkl"),"wb") as of:
    pickle.dump(gs_xgboost_with_missing, of)

In [ ]:
del gs_xgboost_with_missing

### Using sklearn's implementation

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

sk_gbm_classifier = GradientBoostingClassifier(
    n_estimators = 2500,
    subsample = 0.5,
    random_state = Config.get("random_seed"),
    n_iter_no_change = 10
)

gs_sk_gbm = RandomizedSearchCV(
    sk_gbm_classifier,
    param_grid,
    scoring=metrics,
    n_jobs=6,
    cv=cv,
    refit="recall",
    verbose=3,
    return_train_score=True)

In [28]:
gs_sk_gbm.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
gs_sk_gbm.best_params_

In [ ]:
plot_confusion_matrix(y_test, gs_sk_gbm.best_estimator_.predict(X_test), [1,0], normalize=True)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, gs_sk_gbm.best_estimator_.predict_proba(X_test)[:,1],pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")

## XGBoost with Boruta sample

In [ ]:
# We use stochastic gradient boosting (subsample < 1),
# a maximum depth of 2 or 3,
# a very small learning rate combined with early stopping if there's no improvement
param_grid = {
    "colsample_bytree": [0.1, 0.3, 0.5],
    "n_estimators": [100, 400, 800,1600]
}

xgb_classifier_fixed_size = XGBClassifier(
        booster="gbtree",
        scale_pos_weight=scale,
        max_depth=6,
        learning_rate=0.3,
        early_stopping_rounds=10,
        eval_metric=["logloss"],
        n_estimators = 400,
        subsample=0.5, # use half of features for each forest
        seed=Config.get("random_seed"),
        verbose=3)

# Scorer, we choose ROC and set weighted average according to class frequencies
roc = make_scorer(roc_auc_score)

# Creating CV-folds respecting the class frequencies
cv = StratifiedShuffleSplit(n_splits=5, random_state=Config.get("random_seed"))

In [ ]:
gs_unbalanced_xgboost_boruta = GridSearchCV(
    xgb_classifier_fixed_size,
    param_grid,
    scoring={
        "Precision": "precision",
        "ROC_AUC": roc,
        "neglogloss": "neg_log_loss"
    },
    n_jobs=5,
    cv=cv,
    refit="neglogloss",
    verbose=3,
    return_train_score=True)

In [ ]:
gs_unbalanced_xgboost_boruta.fit(X_train_all_relevant,y_train,eval_set=[(X_train_all_relevant, y_train), (X_test_all_relevant, y_test)])

In [ ]:
gs_unbalanced_xgboost_boruta.best_params_

In [ ]:
eval_results_boruta = gs_unbalanced_xgboost_boruta.best_estimator_.evals_result()
epochs = len(eval_results_boruta['validation_0']['logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, eval_results_boruta['validation_0']['logloss'], label='Train')
ax.plot(x_axis, eval_results_boruta['validation_1']['logloss'], label='Test')
logl_min = min(eval_results_boruta['validation_1']['logloss'])
logl_index = eval_results_boruta['validation_1']['logloss'].index(logl_min)
ax.plot((logl_index, logl_index), (0, logl_min), 'k-')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

In [ ]:
predict = gs_unbalanced_xgboost_boruta.predict_proba(X_test_all_relevant)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, gs_unbalanced_xgboost_boruta.predict_proba(X_test_all_relevant)[:,1],pos_label=1)
roc_auc = auc(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.plot(fpr, tpr, lw=1, label='ROC curve (area = %0.2f)' % roc_auc)

In [ ]:
import xgboost
xgboost.plot_importance(gs_unbalanced_xgboost_boruta.best_estimator_.get_booster(),
                        max_num_features=20,
                        show_values=True)

In [ ]:
gs_unbalanced_xgboost_boruta.best_estimator_.feature_importances_

In [ ]:
kdd98_learn_feat_all_relevant.columns.values.tolist()[31]

In [ ]:
importances = pd.DataFrame(gs_unbalanced_xgboost_boruta.best_estimator_.feature_importances_, index=kdd98_learn_feat_all_relevant.columns.values.tolist(), columns=["importance"])

In [ ]:
importances = importances.sort_values(by="importance", ascending=False)

In [ ]:
factor_importance = 100/importances.iloc[0,0]
factor_importance

In [ ]:
importances.importance = importances.importance.map(lambda i: i*factor_importance)

In [ ]:
most_important = importances.head(50)
most_important

In [ ]:
import seaborn as sns

In [ ]:
f, ax = plt.subplots(figsize=(6, 15))
sns.barplot(y=most_important.index, x=most_important.importance)
sns.despine(left=True, bottom=True)